In [1]:
import pyvista as pv
import os, random
import numpy as np
import miniball

# Load single STL

Note: pyvista plotting is very slow (for now) on ncar jupyterhub, so avoid if possible.

In [2]:
stl_dir = '/glade/derecho/scratch/joko/synth-ros/n1000-test-20250226'
id = random.randint(1, 69999)
id = f'{id:06d}'
mesh = pv.read(os.path.join(stl_dir, f'ros-test-{id}.stl'))
mesh

PolyData,Information
N Cells,6354
N Points,3173
N Strips,0
X Bounds,"-1.268e+02, 1.635e+02"
Y Bounds,"-1.626e+02, 1.409e+02"
Z Bounds,"-1.322e+02, 1.476e+02"
N Arrays,0


# Generate N views and save as png

In [ ]:
%%time
# Generate random views of the particle and save as png
save_path = '/glade/derecho/scratch/joko/synth-ros/test-projections'
n_renders = 100
res = 64
bg_color='black'
obj_color='white'
op=0.9

pl = pv.Plotter(off_screen=True, window_size=[res, res])
pl.background_color = bg_color
pl.enable_parallel_projection()
pl.remove_all_lights()

def random_rotate(mesh):
    """
    Rotate rosette in a random orientation
    TODO:
    - fix bug related to the reliance on model attribute
    """
    rotated = mesh.copy()
    deg_x = np.random.randint(1, 360)
    deg_y = np.random.randint(1, 360)
    deg_z = np.random.randint(1, 360)
    rotated_model = rotated.rotate_x(deg_x, inplace=False)
    rotated_model.rotate_y(deg_y, inplace=True)
    rotated_model.rotate_z(deg_z, inplace=True)
    return rotated_model

for i in range(n_renders):
    mesh_rot = random_rotate(mesh)
    pl.add_mesh(mesh_rot, show_edges=None, color = obj_color, opacity=op, name='mesh')
    file_name = f'ros_{i}.png'
    file_path = save_path + '/' + file_name
    # pl.show(auto_close=False, interactive=False, jupyter_backend='none')
    # pl.show(screenshot=file_path, interactive=False, jupyter_backend='none')
    pl.screenshot(file_path, return_img=False)
    # pl.deep_clean()
pl.close()

# Test pyvista calculations

In [3]:
# pyvista calculations for surface area and volume
sa = mesh.area
vol = mesh.volume
print(f'sa = {sa}, vol = {vol}')

sa = 201764.54126171285, vol = 1850305.4527365668


In [6]:
# test minimally bounding sphere and effective density calcs
def calc_mbs(mesh):
    """
    Calculate minimal bounding sphere (mbs)
    """
    mbs = {} # store attributes of sphere as dict

    # use miniball algorithm to find bounding sphere
    mesh_points = np.asarray(mesh.points)
    c, r2 = miniball.get_bounding_ball(mesh_points)
    r = np.sqrt(r2) # r2 = radius squared, r = radius

    mbs['c'] = c # center coordinates as np array
    mbs['r'] = r # radius of sphere as float
    mbs['v'] = (4/3)*np.pi*(r**3)

    return mbs

def calc_rho_eff_ratio(mesh, mbs):
    """
    Calculate effective density ratio
    I.e. volume of rosette / volume of bounding sphere
    """
    rho_eff_ratio = mesh.volume / mbs['v']
    return rho_eff_ratio
mbs = calc_mbs(mesh)
print(mbs)
rho_eff = calc_rho_eff_ratio(mesh, mbs)
print(rho_eff)

{'c': array([ 15.9622283 , -10.79061417,  -0.48734297]), 'r': np.float64(166.21215529677823), 'v': np.float64(19234325.11460258)}
0.09619809594108536
